In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import urllib3
import httpimport
import time
from datetime import timedelta, datetime
from TradingAPI import TradingAPI
from consts import MacioPasswords

with httpimport.github_repo(
    "DZAlpha", "utility", module='influx'
):
    from influx import InfluxClient

In [2]:
bucket = "my-bucket"
org = "ask-org"
token = "kLRh4PbhH-N7I7a0qWCkkClf1zzJFgd_u2AUZoa0_2AYXd-ow-KqAXTvi4UFyO9Dfw4vTuQRD0UZzY9S9xS4_w=="
url="http://bogactwo.ii.uni.wroc.pl:8086"

database_demo = InfluxClient(url=url,bucket=bucket, org=org, token=token)

In [3]:

query_params={'metric': 'model2d','from_time' : '2021-08-29-20:00', 'to_time': '2021-08-29-20:00', 'currency': 'ETHUSDT','fields': ['scaled_forward_return']}
query = database_demo.parse_query(**query_params)
print(query)
preds = database_demo.execute_query(query)
preds.shape[0]

from (bucket:"my-bucket")|> range(start: 2021-08-29T20:00:00Z, stop: 2021-08-29T20:00:01Z)|> filter(fn: (r) => r["_measurement"] == "ETHUSDT")|> filter(fn: (r) => r["metric"] == "model2d")|> filter(fn: (r) => r["_field"] == "scaled_forward_return")


0

In [69]:
class AlphaTrader:
    def __init__(self,
                currency,
                predicted_currency,
                time,
                threshold,
                database):
        self.time=time
        self.currency=currency
        self.predicted_currency=predicted_currency
        self.threshold=threshold
        self.database=database
        self.buy=''
        self.tradingAPI=TradingAPI(MacioPasswords.api_key, MacioPasswords.api_secret)
        self.feedback={}
        
    def fetch_prediction(self):
        query_params={'metric': 'model2d','from_time' : self.time, 'to_time': self.time, 'currency': self.predicted_currency ,'fields': ['scaled_forward_return']}
        query = self.database.parse_query(**query_params)
        while True:
            print(datetime.now())
            
            prediction = self.database.execute_query(query)
            if prediction.shape[0]:
                if float(prediction['_value']) > self.threshold:
                    self.buy='buy'
                elif float(prediction['_value'])> 0:
                    self.buy = 'hold'
                else: 
                    self.buy='sell'
                print(float(prediction['_value']))
                break
            if datetime.now().second > 30:
                break
            time.sleep(1)
            
    
    def attempt_transaction(self):
        for i in range(5):
            if self.buy=='buy':
                self.feedback=self.tradingAPI.buy(self.currency)
            elif self.buy=='hold':
                break
            elif self.buy=='sell': 
                self.feedback=self.tradingAPI.sell(self.currency)
            if self.feedback['status'] == 'FILLED':
                print('succesful',self.feedback['side'],'with price of', self.feedback['price'])
                break
            if self.feedback['status'] =='NO ASSET':
                print('cannot', self.buy, 'because of unsifficient assets')
                break
            time.sleep(1)
        
    def save_transaction(self):
        if self.feedback['status']=='FILLED':
            kwargs={self.buy:self.feedback['price']}
            self.database.save(self.currency, 'Trader', self.time, **kwargs)
            if self.buy=='sell':
                balance=self.tradingAPI.get_asset_balance('USDT')['free']
                kwargs={'balance':balance}
                self.database.save(self.currency, 'Trader', self.time, **kwargs)

In [159]:
Artiz = AlphaTrader('ETHUSDT', 'BTCUSDT', b, 0.02, database_demo)

In [167]:
a = Artiz.tradingAPI.get_asset_balance('USDT')['free']

In [168]:
float(a)

56.99059597

In [117]:
a

'55.91132311'

In [160]:
Artiz.fetch_prediction()

2021-08-30 19:32:00.758849
0.018713924726739856


In [111]:
Artiz.feedback

{'status': 'NO ASSET'}

In [101]:
Artiz.attempt_transaction()

cannot  because of unsifficient assets


In [103]:
Artiz.feedback['status']


'NO ASSET'

In [66]:
Artiz.feedback

{'status': 'FILLED', 'price': 8000}

In [67]:
Artiz.save_transaction()

In [142]:
current_time = pd.Timestamp('2021-08-30T19:10:00+00:00', tz='utc', unit='ms') + timedelta(minutes=5)

In [158]:
b = str(current_time)
b = parse_date('2021-08-30T19:20:00+00:00')
b

'2021-08-30-19:25'

In [131]:
def parse_date(date):
    date_str = str(pd.Timestamp(date,tz='utc', unit='ms') + timedelta(minutes=5))
    return date_str[:10]+'-'+date_str[11:16]
parse_date('2018-01-01T00:00:00+00:00')

'2018-01-01-00:05'

In [83]:
input_time = pd.Timestamp(
        datetime(
            current_time.year,
            current_time.month,
            current_time.day,
            current_time.hour,
            current_time.minute
        ),
        tz='utc',
        unit='ms'
    )

In [86]:
'{}-{}-{}-{}:{}'.format(a.year, a.month, a.day, a.hour, a.minute)

'2018-1-1-0:5'

In [93]:
b= '2018-01-01T00:00:00+00:00'

In [96]:
dt = b[:10]+'-'+b[11:16]

In [97]:
dt

'2018-01-01-00:00'